In [1]:
import pandas as pd

with open('C:/Users/win8/PycharmProjects/textmining/venv/1111stop.txt', encoding='utf8') as file:
    stopWord_list = [k.strip() for k in file.readlines()]

train_df = pd.read_csv('C:/Users/win8/PycharmProjects/textmining/venv/train.txt',sep='\t', header=None,)
train_df = train_df.astype(str)
train_df.shape

(2006, 2)

In [2]:
import jieba
import time

train_df.columns = ['分类', '文章']
stopword_list = [k.strip() for k in open('C:/Users/win8/Desktop/1111stop.txt', encoding='utf8').readlines() if k.strip() != '']
cutWords_list = []
i = 0
startTime = time.time()
for article in train_df['文章']:
    cutWords = [k for k in jieba.cut(article) if k not in stopword_list]
    i += 1
    if i % 1000 == 0:
        print('前%d篇文章分词共花费%.2f秒' %(i, time.time()-startTime))
    cutWords_list.append(cutWords)
with open('C:/Users/win8/PycharmProjects/textmining/venv/cutWords_list.txt', 'w',encoding='utf-8') as file: 
    for cutWords in cutWords_list:
        file.write(' '.join(cutWords) + '\n')
with open('C:/Users/win8/PycharmProjects/textmining/venv/cutWords_list.txt',encoding='utf-8') as file:
    cutWords_list = [k.split() for k in file.readlines()]

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\win8\AppData\Local\Temp\jieba.cache
Loading model cost 1.213 seconds.
Prefix dict has been built succesfully.


前1000篇文章分词共花费42.50秒
前2000篇文章分词共花费71.21秒


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf1 = TfidfVectorizer(cutWords_list)
X = tf1.fit_transform(train_df['文章'])
print(X.shape)

(2006, 107516)


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(cutWords_list, min_df=10, max_df=0.6)
X = tfidf.fit_transform(train_df['文章'])
print(X.shape)

(2006, 11578)


In [5]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

train_df = pd.read_csv('C:/Users/win8/PycharmProjects/textmining/venv/train.txt', sep='\t', header=None)
train_df = train_df.astype(str)
labelEncoder = LabelEncoder()
y = labelEncoder.fit_transform(train_df[0].values)
y.shape

(2006,)

In [6]:
train_df[0].value_counts()

互联网    252
娱乐     252
财经     251
零售     251
体育     250
资讯     250
电商     250
军事     250
Name: 0, dtype: int64

In [7]:
import time  
from sklearn import metrics  
import pickle as pickle  
import pandas as pd
from sklearn import metrics

In [8]:
#降维
from sklearn.feature_selection import SelectKBest,chi2
X=SelectKBest(chi2,k=500).fit_transform(X,y)
X.shape

(2006, 500)

In [41]:
#划分训练测试集
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2)
train_x.shape

(1604, 500)

In [42]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=9,min_samples_split=3)
model.fit(train_x, train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=3,
            min_weight_fraction_leaf=0.0, n_estimators=9, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [43]:
#模型评估模块
predict = model.predict(test_x)
precision = metrics.precision_score(test_y, predict, average='macro')
recall = metrics.recall_score(test_y, predict, average='macro')
print('precision: %.2f%%, recall: %.2f%%' % (100 * precision, 100 * recall))
accuracy = metrics.accuracy_score(test_y, predict)
print('accuracy: %.2f%%' % (100 * accuracy))

precision: 98.04%, recall: 97.95%
accuracy: 98.01%


In [44]:
#绘制混淆矩阵
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(test_y, predict), 
             columns=labelEncoder.classes_, 
             index=labelEncoder.classes_)

,互联网,体育,军事,娱乐,电商,财经,资讯,零售
互联网,51,0,0,0,0,0,0,0
体育,0,63,0,1,0,0,0,0
军事,0,0,46,0,0,0,0,0
娱乐,0,0,0,49,0,0,0,0
电商,1,0,0,0,55,0,0,0
财经,1,0,0,1,2,40,0,2
资讯,0,0,0,0,0,0,47,0
零售,0,0,0,0,0,0,0,43


In [45]:
#保存模型
import pickle
with open('rfclassifier.model', 'wb') as file:
    save = {
        'labelEncoder' : labelEncoder,
        'tfidfVectorizer' : tfidf,
        'logistic_model' : model
    }
    pickle.dump(save, file)

In [295]:
#交叉验证
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
parameters = { 'C':[1, 2, 4], 'max_iter':[10,50,100]}
svc = SVC(gamma='auto',probability=True,kernel='rbf')
model = GridSearchCV(svc, parameters, n_jobs=-1,cv=5)
model.fit(train_x, train_y)
cv_result = pd.DataFrame.from_dict(model.cv_results_)   
print('The parameters of the best model are: ')
print(model.best_params_)

The parameters of the best model are: 
{'C': 4, 'max_iter': 50}


C:\Users\win8\PycharmProjects\weixin\venv\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=50).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\win8\PycharmProjects\weixin\venv\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\win8\PycharmProjects\weixin\venv\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\win8\PycharmProjects\weixin\venv\lib\site-packages\sklearn\utils\depreca